# 1. load model

In [1]:
import json
import pytorch_lightning as pl
from einops import rearrange
import torch
import torchaudio
from torch.utils.data import DataLoader
import safetensors
from safetensors.torch import load_file
import random

import shutil
from collections import OrderedDict
from datetime import datetime
import os
import torch.nn.functional as F

from stable_audio_tools import create_model_from_config, replace_audio, save_audio
from stable_audio_tools.data.dataset import VideoFeatDataset, VideoFeatDataset_VL, collation_fn
from stable_audio_tools.training.training_wrapper import DiffusionCondTrainingWrapper
from stable_audio_tools.inference.generation import generate_diffusion_cond, generate_diffusion_cond_from_path


# model_config_file = './stable_audio_tools/configs/model_config_ss30.json'
model_config_file = './stable_audio_tools/configs/model_config_vl30.json'
model_weight = './weight/epoch=60-step=2818.safetensors'


with open(model_config_file) as f:
    model_config = json.load(f)
    sample_rate = model_config["sample_rate"]
    sample_size = model_config["sample_size"]
    fps = model_config["fps"]
    variable_length = model_config["variable_length"]
    force_channels = "stereo" if model_config["audio_channels"] == 2 else "mono"


state_dict = load_file(model_weight)
model = create_model_from_config(model_config)
model.load_state_dict(state_dict, strict=True)

sample_size, sample_rate, sample_size/sample_rate, variable_length, fps, force_channels

/home/chengxin/chengxin/anaconda3/envs/stableaudio/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<module 'flash_attn_2_cuda' from '/home/chengxin/chengxin/anaconda3/envs/stableaudio/lib/python3.8/site-packages/flash_attn_2_cuda.cpython-38-x86_64-linux-gnu.so'>


(1323000, 44100, 30.0, 30, 8, 'stereo')

# 2. Sample

In [2]:
from moviepy.editor import VideoFileClip

device = 0
conditioning = {
    'feature': ['./asset/demo_video.mp4']
}
video_clip = VideoFileClip(conditioning['feature'][0])
seconds_total = int(video_clip.duration)
seconds_total = 10

output = generate_diffusion_cond(
    model = model.to(device),
    steps=150,
    cfg_scale=7,
    conditioning=conditioning,
    sample_size=int(sample_rate*seconds_total),
    batch_size=len(conditioning['feature']),
    sigma_min=0.3,
    sigma_max=500,
    sampler_type="dpmpp-3m-sde",
    device=device
)

audio_path = "./asset/demo_audio.wav"
waveform = output[0:1,...,:int(seconds_total*sample_rate)]
# print(output.shape, output[idx:idx+1].shape, waveform.shape)
save_audio(waveform, audio_path, sample_rate)
waveform

346729609
Extracting features from video:./asset/demo_video.mp4


100%|██████████| 150/150 [00:03<00:00, 48.96it/s]


tensor([[[-0.0644,  0.0689, -0.0363,  ...,  0.0074,  0.0036, -0.0104],
         [-0.0644,  0.0689, -0.0363,  ...,  0.0078,  0.0041, -0.0100]]],
       device='cuda:0')